In [21]:
!pip install kaggle

In [22]:
import os

In [23]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [24]:
!chmod 600 /content/kaggle.json

chmod: cannot access '/content/kaggle.json': No such file or directory


In [25]:
!kaggle datasets download -d marquis03/flower-classification

Dataset URL: https://www.kaggle.com/datasets/marquis03/flower-classification
License(s): Apache 2.0
flower-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [27]:
!nvidia_smi

/bin/bash: line 1: nvidia_smi: command not found


In [28]:
!nvidia-smi

Tue May  7 08:43:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0              27W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [29]:
import torch
torch.cuda.is_available()

True

In [30]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [31]:
!unzip '/content/flower-classification.zip' -d '/content/flower_data'

Archive:  /content/flower-classification.zip
replace /content/flower_data/classname.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [32]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [33]:
BATCH_SIZE=4
IMAGE_SIZE=256
CHANNELS=3

In [34]:
train_ds, test_ds = tf.keras.preprocessing.image_dataset_from_directory("/content/flower_data/train",
                                                              validation_split=0.1,
                                                              subset="both",
                                                              seed=162,
                                                              shuffle=True,
                                                              image_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size=BATCH_SIZE)

Found 13642 files belonging to 14 classes.
Using 12278 files for training.
Using 1364 files for validation.


In [35]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory("/content/flower_data/val",
                                                              seed=162,
                                                              shuffle=True,
                                                              image_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size=BATCH_SIZE)

Found 98 files belonging to 14 classes.


In [36]:
classes = train_ds.class_names

In [37]:
print(classes)

['astilbe', 'bellflower', 'black_eyed_susan', 'calendula', 'california_poppy', 'carnation', 'common_daisy', 'coreopsis', 'dandelion', 'iris', 'rose', 'sunflower', 'tulip', 'water_lily']


In [38]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [39]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [40]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [41]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [42]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(classes)

In [43]:
n_classes

14

In [44]:
model_a = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

In [45]:
model_a.build(input_shape=input_shape)

In [46]:
model_a.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history_a = model_a.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=validation_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
3070/3070 [==============================] - 141s 43ms/step - loss: 1.9307 - accuracy: 0.3063 - val_loss: 1.7289 - val_accuracy: 0.3776
Epoch 2/50
3070/3070 [==============================] - 121s 39ms/step - loss: 1.4781 - accuracy: 0.4765 - val_loss: 1.1944 - val_accuracy: 0.5306
Epoch 3/50
3070/3070 [==============================] - 127s 41ms/step - loss: 1.3243 - accuracy: 0.5328 - val_loss: 1.2157 - val_accuracy: 0.5918
Epoch 4/50
3070/3070 [==============================] - 121s 39ms/step - loss: 1.2263 - accuracy: 0.5692 - val_loss: 1.0714 - val_accuracy: 0.6531
Epoch 5/50
3070/3070 [==============================] - 125s 41ms/step - loss: 1.1725 - accuracy: 0.5911 - val_loss: 1.2195 - val_accuracy: 0.5714
Epoch 6/50
3070/3070 [==============================] - 127s 41ms/step - loss: 1.1268 - accuracy: 0.6100 - val_loss: 0.9646 - val_accuracy: 0.6633
Epoch 7/50
3070/3070 [==============================] - 121s 39ms/step - loss: 1.0825 - accuracy: 0.6284 - val_loss: 0

In [ ]:
model_a.save('/content/model_a')

In [ ]:
model_b = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(n_classes, activation='softmax'),
])

In [ ]:
model_b.build(input_shape=input_shape)

In [ ]:
model_b.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history_b = model_b.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=validation_ds,
    verbose=1,
    epochs=50,
)

In [ ]:
model_b.save('/content/model_b')